#18L-1215, 18L-1014 and 18L-1121

#####Data Alteration

In [ ]:
import numpy as np
import pandas as pd
import random
room_capacity=pd.read_csv('Capacity.csv', header=None)
Slots=pd.read_csv('DaysSlots.csv', header=None)
R_data=pd.read_csv('R_Data.csv')
grouped_df = R_data.groupby(["SID\CID"])
grouped_and_summed = grouped_df.sum()
grouped_and_summed = grouped_and_summed.reset_index()
StdInCourse = grouped_and_summed.sum(axis=0)
Courses_of_Sdd = grouped_and_summed.sum(axis=1)
StdInCourse=StdInCourse.drop(["SID\CID"], axis=0)
NoOfStdInCourse=StdInCourse.to_numpy()
r=room_capacity.iloc[: , 1:]
room_cap=r.to_numpy().flatten()
Arr=grouped_and_summed.drop(['SID\CID'], axis=1)
N=Arr.to_numpy().astype(int)
Student_courses=[]
for i in N:
    Array=[]
    count=0
    for j in i:
        if(j==1):
            Array.append(count+1)
        count+=1;
    Student_courses.append(Array)
A=Slots.to_numpy()
B=room_capacity.to_numpy()
num_rows, num_cols = B.shape
cols=grouped_and_summed.columns.to_numpy()[1:]
courses=cols.astype(int)
NoOfTimeSlots=A[0][1]*A[1][1]
NoOfRooms=num_rows

#####Fitness Function

In [ ]:
import numpy as np
def Fitness_Function(Solution):
    Clashes_Count=0      # Finding number of students who have clashes
    for s in Student_courses:
        arr=[]
        for i in range(len(s)):
            arr.append(Solution[s[i]-1])
        contains_duplicates = any(arr. count(element) > 1 for element in arr)
        if contains_duplicates==True:
            Clashes_Count+=1
    CoursesInSlot=[[] for j in range(NoOfTimeSlots)]
    for i in range(len(Solution)):
        CoursesInSlot[Solution[i]].append(i+1)
    n=NoOfStdInCourse.copy()
    StudentsLeftSpaired=0       #Finding number of solutions who could not give exams on their exam slot
    for s in CoursesInSlot:
        room_no=0
        for i in range(len(s)):
            students=n[s[i]-1]
            while students > 0 and room_no < NoOfRooms:
                students-=room_cap[room_no]
                room_no+=1
            if (room_no == NoOfRooms):
                StudentsLeftSpaired+=students
    if Clashes_Count == 0 and StudentsLeftSpaired == 0:   # No clashes, no left over students
        return 1
    print('Number of students having clashes')
    print(Clashes_Count)
    print('Number of students who are not allocated in rooms')
    print(StudentsLeftSpaired)
    return 1/(StudentsLeftSpaired+Clashes_Count)

#####Neighboorhood Operator:

In [ ]:
import random
def neighbor(Sol):
    ans=Sol.copy()
    index=random.randint(0,len(courses)-1)
    ans[index]=random.randint(0,NoOfTimeSlots-1)
    return ans

Crossover Function:

In [ ]:
def crossover(Solutions):  #top best solutions
    for i in range(0,500):
        sol1=Solutions[random.randint(0,len(Solutions)-1)]
        sol2=Solutions[random.randint(0,len(Solutions)-1)]
        on_point=random.randint(0,NoOfTimeSlots-1)
        arr=[]
        arr1=[]
        for j in range(0,on_point):
            arr.append(sol1[j])
            arr1.append(sol2[j])
        for j in range(on_point,len(Solutions[i])):
            arr.append(sol2[j])
            arr1.append(sol1[j])
        arr = mutation(arr)
        arr1 = mutation(arr1)
        Solutions.append(arr)
        Solutions.append(arr1)
    return Solutions

#####Mutation Operation:

In [ ]:
def mutation(Solution):
    prob = random.uniform(0,1)
    if prob < 0.1:
        index=random.randint(0,len(courses)-1)
        Solution[index]=random.randint(0,NoOfTimeSlots-1)
    return Solution

#####Local Search

In [ ]:
#generating 100 neighbours
def Local_Search(Solution):
    best=Fitness_Function(Solution)
    neighbourHood=[]

    for i in range(1000):
        print('Local Search Best', i , 'Fitness So Far ', best,' solution',Solution)
        if best > 0.95:
            break
        for i in range(1000):#finding 100 solutions
            sol=neighbor(Solution)
            neighbourHood.append((Fitness_Function(sol),sol))
        neighbourHood.sort()
        neighbourHood.reverse()

        newScore=neighbourHood[0][0]
        best_Sol=neighbourHood[0][1]

        if newScore > best:
            Solution.clear()
            Solution=best_Sol.copy()
            best=newScore
        neighbourHood.clear()
    return best_Sol

#####Genetic Algorithm:


In [ ]:
def genetic_algo(Sols,generations):
    for generation_count in range(generations):
        generation=[]
        for i in range(len(Sols)):
            generation.append([Fitness_Function(Sols[i]),Sols[i]])
        generation.sort()
        generation.reverse()
        gen=[]
        print('Genetic Algo: Best ',generation_count,' Fitness So Far ', generation[0][0],' solution',generation[0][1])
        if generation[0][0] == 1:
            break
        for j in range(100):
            gen.append(generation[j][1])
        new_generation=crossover(gen)
        Sols=new_generation
    return generation[0][1]

####Creating population of 1000:

In [ ]:
Sols=[]
for j in range(1000):
    Solution=[]
    for i in range(len(courses)):
        Solution.append(random.randint(0,NoOfTimeSlots-1))
    Sols.append(Solution)

#####Finding Best Time Table using Genetic Algorithm and Local Search

In [ ]:
Solution=genetic_algo(Sols,1)
Solution=Local_Search(Solution)


Genetic Algo: Best  0  Fitness So Far  0.005847953216374269  solution [3, 18, 13, 7, 19, 17, 1, 15, 8, 16, 16, 11, 7, 4, 9, 1, 14, 19, 8, 15, 12, 9, 6, 18, 5, 9, 4, 6, 16, 12, 0, 7, 1, 3, 10, 18, 5, 2, 9, 9, 16, 6, 4, 19, 9, 7, 17, 0, 6, 1, 11, 7, 4, 15, 17, 2, 3, 3, 4, 7, 10, 3, 9, 19, 14, 4, 6, 10, 14, 11, 5]
Local Search Best 0 Fitness So Far  0.005847953216374269  solution [3, 18, 13, 7, 19, 17, 1, 15, 8, 16, 16, 11, 7, 4, 9, 1, 14, 19, 8, 15, 12, 9, 6, 18, 5, 9, 4, 6, 16, 12, 0, 7, 1, 3, 10, 18, 5, 2, 9, 9, 16, 6, 4, 19, 9, 7, 17, 0, 6, 1, 11, 7, 4, 15, 17, 2, 3, 3, 4, 7, 10, 3, 9, 19, 14, 4, 6, 10, 14, 11, 5]
Local Search Best 1 Fitness So Far  0.006944444444444444  solution [3, 18, 13, 13, 19, 17, 1, 15, 8, 16, 16, 11, 7, 4, 9, 1, 14, 19, 8, 15, 12, 9, 6, 18, 5, 9, 4, 6, 16, 12, 0, 7, 1, 3, 10, 18, 5, 2, 9, 9, 16, 6, 4, 19, 9, 7, 17, 0, 6, 1, 11, 7, 4, 15, 17, 2, 3, 3, 4, 7, 10, 3, 9, 19, 14, 4, 6, 10, 14, 11, 5]
Local Search Best 2 Fitness So Far  0.008333333333333333  solution

In [ ]:
sols=[3, 18, 13, 13, 19, 17, 1, 12, 8, 18, 16, 11, 15, 10, 11, 0, 14, 19, 8, 2, 12, 13, 6, 18, 5, 9, 17, 6, 16, 12, 0, 7, 1, 3, 10, 18, 5, 2, 9, 9, 16, 1, 4, 5, 9, 7, 17, 0, 12, 1, 8, 7, 4, 15, 2, 15, 15, 7, 14, 9, 10, 3, 19, 11, 14, 4, 6, 17, 16, 11, 5]
print(Fitness_Function(sols))

Number of students having clashes
1
Number of students who are not allocated in rooms
0
1.0
